# Data Extract Testing

In [95]:
import pandas as pd
import pandas_profiling
import os
# Set the styling for dataframes
dfstyles = [
    dict(selector="caption",
         props=[("font-size", "120%"), ("color", "black"),
                ("text-align", "center")])
]
from IPython.display import display, HTML

In [96]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<b>NOTE</b>:The code for this notebook is, by default, hidden. To display the code cells, click <a href="javascript:code_toggle()">here</a>.\r\n''')

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem-Statement" data-toc-modified-id="Problem-Statement-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem Statement</a></span></li><li><span><a href="#Test-Data" data-toc-modified-id="Test-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test Data</a></span></li><li><span><a href="#Test-Cases" data-toc-modified-id="Test-Cases-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test Cases</a></span><ul class="toc-item"><li><span><a href="#Results-with-@Columnlist-=-NULL-and--@Columnlist-=-0-should-be-identical" data-toc-modified-id="Results-with-@Columnlist-=-NULL-and--@Columnlist-=-0-should-be-identical-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Results with @Columnlist = NULL and  @Columnlist = 0 should be identical</a></span></li><li><span><a href="#Columns-in-@Columnlist-=-0-should-be-identical-to-&quot;Name&quot;-column-of-@Columnlist-=-1" data-toc-modified-id="Columns-in-@Columnlist-=-0-should-be-identical-to-&quot;Name&quot;-column-of-@Columnlist-=-1-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Columns in @Columnlist = 0 should be identical to "Name" column of @Columnlist = 1</a></span></li><li><span><a href="#Column-names-have-been-changed,-check-Aicraft-Attribute-is-fixed" data-toc-modified-id="Column-names-have-been-changed,-check-Aicraft-Attribute-is-fixed-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Column names have been changed, check Aicraft Attribute is fixed</a></span></li><li><span><a href="#Columns-have-changed-(original-SP-to-new)-but-the-data-should-be-exact" data-toc-modified-id="Columns-have-changed-(original-SP-to-new)-but-the-data-should-be-exact-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Columns have changed (original SP to new) but the data should be exact</a></span></li><li><span><a href="#Previous-column-headers-file-should-match-the-new-headers-file" data-toc-modified-id="Previous-column-headers-file-should-match-the-new-headers-file-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Previous column headers file should match the new headers file</a></span></li><li><span><a href="#Final-output-file-should-contain-45-days-of-data-(Actual-Departure-Date)" data-toc-modified-id="Final-output-file-should-contain-45-days-of-data-(Actual-Departure-Date)-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Final output file should contain 45 days of data (Actual Departure Date)</a></span></li></ul></li></ul></div>

## Problem Statement
* The customer would like a reference file, mapping the system names with the friendly names in the data extract.
* There is an issue with the current extract, whereby only the attribute code is output for dimensions meaning there are duplicate headers; the attribute code and attribute column names should be ":" separated, if the parameter is a key. e.g. Aircraft:Registration.

## Test Data
12 Test result files were produced by saving the output of the original SP: GetOperatorHistoricData and the updated SP.
The original SP was run for UPS, CPA and SIA in the following configuration:

`EXEC   @return_value = [MartFuel].[GetOperatorHistoricData]
		@CustomerCode = N'<OPERATOR NAME>
		@OperatorCode = NULL,
		@Fields = NULL,
		@FieldsToOrderBy = N'ActualDepartureDatetime',
		@FriendlyNames = 0,
		@DateRange = NULL,
		@DateFrom = NULL,
		@DateTo = NULL,
		@TopN = 10,
		@Columnlist = 0`


The new SP was run with the same configuration as above, but with an additional parameter added. <BR> Values of @parametermap was varied as below:

`@Columnlist = NULL
 @Columnlist = 0
 @Columnlist = 1`

In [97]:
# Define sorting
sortcols = ['ActualDepartureDatetime', 'Aircraft:Registration']

# Source DIR
sourcedir = 'H:/UPS/UPS_Extract_SP_Prod/Extract_Testing/'

# Import and sort
New_SP_0_Param_CPA_df = pd.read_csv(sourcedir +
                                    'New_SP_0_Param_CPA.csv').sort_values(
                                        by=sortcols)
New_SP_0_Param_SIA_df = pd.read_csv(sourcedir +
                                    'New_SP_0_Param_SIA.csv').sort_values(
                                        by=sortcols)
New_SP_0_Param_UPS_df = pd.read_csv(sourcedir +
                                    'New_SP_0_Param_UPS.csv').sort_values(
                                        by=sortcols)
New_SP_1_Param_CPA_df = pd.read_csv(sourcedir + 'New_SP_1_Param_CPA.csv')
New_SP_1_Param_SIA_df = pd.read_csv(sourcedir + 'New_SP_1_Param_SIA.csv')
New_SP_1_Param_UPS_df = pd.read_csv(sourcedir + 'New_SP_1_Param_UPS.csv')
New_SP_NULL_Param_CPA_df = pd.read_csv(
    sourcedir + 'New_SP_NULL_Param_CPA.csv').sort_values(by=sortcols)
New_SP_NULL_Param_SIA_df = pd.read_csv(
    sourcedir + 'New_SP_NULL_Param_SIA.csv').sort_values(by=sortcols)
New_SP_NULL_Param_UPS_df = pd.read_csv(
    sourcedir + 'New_SP_NULL_Param_UPS.csv').sort_values(by=sortcols)
Original_SP_CPA_df = pd.read_csv(sourcedir + 'Original_SP_CPA.csv')
Original_SP_SIA_df = pd.read_csv(sourcedir + 'Original_SP_SIA.csv')
Original_SP_UPS_df = pd.read_csv(sourcedir + 'Original_SP_UPS.csv')
Mapping_df = pd.read_csv(sourcedir + '20191015-20191127 - Mapping.csv')
finaldf = pd.read_csv(sourcedir + 'Data_2020-01-25_2020-03-10.csv', low_memory=False)

In [98]:
pd.DataFrame(os.listdir(sourcedir), columns=['Test file names'])

,Test file names
0,New_SP_0_Param_CPA.csv
1,New_SP_0_Param_SIA.csv
2,New_SP_0_Param_UPS.csv
3,New_SP_1_Param_CPA.csv
4,New_SP_1_Param_SIA.csv
5,New_SP_1_Param_UPS.csv
6,New_SP_NULL_Param_CPA.csv
7,New_SP_NULL_Param_SIA.csv
8,New_SP_NULL_Param_UPS.csv
9,Original_SP_CPA.csv


In [99]:
# Remove query results at the bottom of each file
New_SP_1_Param_CPA_df.drop(New_SP_1_Param_CPA_df.tail(2).index, inplace=True)
New_SP_1_Param_UPS_df.drop(New_SP_1_Param_UPS_df.tail(2).index, inplace=True)
New_SP_1_Param_SIA_df.drop(New_SP_1_Param_SIA_df.tail(2).index, inplace=True)

## Test Cases

### Results with @Columnlist = NULL and  @Columnlist = 0 should be identical
Compare `New_SP_0_Param_<XXX>.csv` with `New_SP_NULL_Param_<XXX>.csv`
<BR> Where `<XXX>` = Operator ICAO Code

In [100]:
merged = New_SP_0_Param_CPA_df.merge(New_SP_NULL_Param_CPA_df, indicator=True, how='outer', how = key)
print("CPA - Difference Rows:",merged[merged['_merge'] != 'both'].shape[0])

merged = New_SP_0_Param_SIA_df.merge(New_SP_NULL_Param_SIA_df, indicator=True, how='outer')
print("SIA - Difference Rows:",merged[merged['_merge'] != 'both'].shape[0])

merged = New_SP_0_Param_UPS_df.merge(New_SP_NULL_Param_UPS_df, indicator=True, how='outer')
print("UPS - Difference Rows:",merged[merged['_merge'] != 'both'].shape[0])

CPA - Difference Rows: 0
SIA - Difference Rows: 0
UPS - Difference Rows: 0


In [116]:
merged[merged['_merge'] == 'both'].T

,0,1,2,3,4,5,6,7,8,9,...,820,821,822,823,824,825,826,827,828,829
Name,Aircraft:CustomerAircraftDesign,Aircraft:Registration,Aircraft:IATACode,Aircraft:Configuration,FlightNumber:Code,FlightNumber:Name,UpliftFuelVolume,OutFuelMass,OffFuelMass,OnFuelMass,...,metar_act_dep_OverallImpactingConditionsClassi...,metar_act_dep_SeverityOfLocalWeatherConditions...,metar_act_dep_temp_c,metar_act_dep_ThunderstormsWithin50MilesClassi...,metar_act_dep_visibility_statute_mi,metar_act_dep_VisibilityInStatuteMilesClassifi...,metar_act_dep_wind_dir_degrees,metar_act_dep_wind_speed_kt,metar_act_dep_WindSpeedInKnotsClassification,EUETSUpliftFuel
FriendlyName,AC Type Aircraft Type,AC Type Tail Number,AC Type IATACode,AC Type Configuration,Flt Number Code,Flt Number Name,Uplift Gal ACARS,Out Fuel,Off Fuel,On Fuel,...,Actual Departure Airport Overall Impact Cl.,Actual Departure Airport Severity Cl.,Actual Departure Airport Temperature,Actual Departure Airport Thunderstorm Cl.,Actual Departure Airport Visibility,Actual Departure Airport Visibility Cl.,Actual Departure Airport Wind Direction,Actual Departure Airport Wind Speed,Actual Departure Airport Wind Speed Cl.,EUETS Uplift Fuel
_merge,both,both,both,both,both,both,both,both,both,both,...,both,both,both,both,both,both,both,both,both,both


### Columns in @Columnlist = 0 should be identical to "Name" column of @Columnlist = 1
Compare columns of `New_SP_0_Param_<XXX>.csv` with Name column of `New_SP_1_Param_<XXX>.csv`
<BR> Where `<XXX>` = Operator ICAO Code

In [101]:
print("CPA - differences: ",len(list(set(New_SP_1_Param_CPA_df['Name'].sort_values().to_list())-set(New_SP_0_Param_CPA_df.columns.to_list()))))
print("SIA - differences: ",len(list(set(New_SP_1_Param_SIA_df['Name'].sort_values().to_list())-set(New_SP_0_Param_SIA_df.columns.to_list()))))
print("UPS - differences: ",len(list(set(New_SP_1_Param_UPS_df['Name'].sort_values().to_list())-set(New_SP_0_Param_UPS_df.columns.to_list()))))

CPA - differences:  0
SIA - differences:  0
UPS - differences:  0


### Column names have been changed, check Aicraft Attribute is fixed 
Compare Aicraft* columns `Original_SP_UPS.csv` with `New_SP_0_Param_UPS.csv`

In [102]:
origaircraft = Original_SP_CPA_df.sort_values(['ActualDepartureDatetime', 'Aircraft.1']).head(2).filter(regex='^Aircraft').T.reset_index()
newaircraft = New_SP_0_Param_CPA_df.sort_values(['ActualDepartureDatetime', 'Aircraft:Registration']).head(2).filter(regex='^Aircraft').T.reset_index()
merged = origaircraft.merge(newaircraft, indicator=False, how='outer', left_index = True, right_index = True,suffixes = ['_old','_new'])

In [103]:
merged

,index_old,0_old,1_old,index_new,0_new,1_new
0,Aircraft,A320,747-400,Aircraft:CustomerAircraftDesign,A320,747-400
1,Aircraft.1,B-HSU,B-HKV,Aircraft:Registration,B-HSU,B-HKV
2,Aircraft.2,NaN,74K,Aircraft:IATACode,NaN,74K
3,Aircraft.3,NaN,359,Aircraft:Configuration,NaN,359
4,AircraftRegistration2,B-HSU,B-HKV,AircraftRegistration2:Code,B-HSU,B-HKV
5,AircraftRegistration2.1,B-HSU,B-HKV,AircraftRegistration2:Name,B-HSU,B-HKV
6,AircraftType,NaN,NaN,AircraftType:CustomerAircraftDesign,NaN,NaN
7,AircraftType.1,NaN,NaN,AircraftType:Registration,NaN,NaN
8,AircraftType.2,NaN,NaN,AircraftType:IATACode,NaN,NaN
9,AircraftType.3,NaN,NaN,AircraftType:Configuration,NaN,NaN


### Columns have changed (original SP to new) but the data should be exact
Compare data `Original_SP_<XXX>.csv` with `New_SP_0_Param_<XXX>.csv`
<BR> Where `<XXX>` = Operator ICAO Code

In [104]:
# Columns have been fixed in the new version but the data should be the same so changing column names to new ones
Original_SP_CPA_df.columns = New_SP_0_Param_CPA_df.columns
Original_SP_SIA_df.columns = New_SP_0_Param_SIA_df.columns
Original_SP_UPS_df.columns = New_SP_0_Param_UPS_df.columns

In [105]:
merged = Original_SP_CPA_df.merge(New_SP_0_Param_CPA_df, indicator=True, how='outer')
print("CPA - Difference Rows:",merged[merged['_merge'] != 'both'].shape[0])
merged = Original_SP_SIA_df.merge(New_SP_0_Param_SIA_df, indicator=True, how='outer')
print("SIA - Difference Rows:",merged[merged['_merge'] != 'both'].shape[0])
merged = Original_SP_UPS_df.merge(New_SP_0_Param_UPS_df, indicator=True, how='outer')
print("UPS - Difference Rows:",merged[merged['_merge'] != 'both'].shape[0])

CPA - Difference Rows: 0
SIA - Difference Rows: 0
UPS - Difference Rows: 0


### Previous column headers file should match the new headers file
Compare data `New_SP_1_Param_UPS.csv` with `20191015-20191127 - Mapping.csv`

In [106]:
merged = New_SP_1_Param_UPS_df.merge(Mapping_df, indicator=True, how='outer')
print("UPS Mapping - Difference Rows:",merged[merged['_merge'] != 'both'].shape[0])

UPS Mapping - Difference Rows: 0


### Final output file should contain 45 days of data (Actual Departure Date)

In [107]:
mindate = min(finaldf['ActualDepartureDatetime'].astype('datetime64[ns]'))
maxdate = max(finaldf['ActualDepartureDatetime'].astype('datetime64[ns]'))

print("Earliest Date in file:", mindate)
print("Latest Date in file:", maxdate)
print("Date difference in file", maxdate - mindate )

Earliest Date in file: 2020-01-25 00:03:00
Latest Date in file: 2020-03-10 23:52:00
Date difference in file 45 days 23:49:00


In [111]:
finaldf['key'] = finaldf['ActualDepartureDatetime'] + finaldf['Aircraft:Registration']

In [112]:
finaldf['key']

0        2020-01-25 00:03:00N303UP
1        2020-01-25 00:04:00N337UP
2        2020-01-25 00:15:00N611UP
3        2020-01-25 00:46:00N350UP
4        2020-01-25 00:49:00N343UP
                   ...            
23150    2020-03-10 23:43:00N143UP
23151    2020-03-10 23:45:00N157UP
23152    2020-03-10 23:45:00N581UP
23153    2020-03-10 23:46:00N172UP
23154    2020-03-10 23:52:00N174UP
Name: key, Length: 23155, dtype: object